<a href="https://colab.research.google.com/github/spshreya/genai/blob/main/IMDB_movie_review_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import kagglehub
import pandas as pd
import re
import string, time
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
nltk.download('stopwords')
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 255)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [4]:
movie_df = pd.read_csv(path + "/IMDB Dataset.csv")

In [5]:
movie_df.shape

(50000, 2)

#### NLP foundations exploration
can skip directly to Text classification

In [ ]:
movie_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
movie_df = movie_df[:100]

In [ ]:
movie_df['review'].loc[3]

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

In [ ]:
movie_df['review'] = movie_df['review'].str.lower()

In [ ]:
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
movie_df['review'] = movie_df['review'].apply(remove_html_tags)

In [ ]:
movie_df['review'].loc[3]

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [ ]:
text1 = 'hello world https://www.google.com'
remove_urls(text1)

'hello world '

In [ ]:
exclude = string.punctuation
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_punctuations(text):
    for char in exclude:
        text = text.replace(char, '')
    return text

In [ ]:
start = time.time()
text2= '`hello world!`'
time2 = time.time() - start
remove_punctuations(text2)
print(time2*50000)

5.018711090087891


In [ ]:
def remove_punctuations_new(text):
    return text.translate(str.maketrans('', '', exclude))

In [ ]:
start = time.time()
text2= '`hello world!`'
time2 = time.time() - start
remove_punctuations_new(text2)
print(time2*50000)

8.082389831542969


In [ ]:
chat_words = {
    'AFAIK':'As Far As I Know',
    'AFK':'Away From Keyboard',
    'ASAP':'As Soon As Possible',
    'ATK':'At The Keyboard',
    'ATM':'At The Moment',
}

In [ ]:
def chat_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [ ]:
chat_conversion('do this asap')

'do this As Soon As Possible'

In [ ]:
incorrect_text = 'thiis speling is wrogn'
textBlb = TextBlob(incorrect_text)
textBlb.correct().string

'this spelling is wrong'

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
def stopword_removal(text):
    new_list = []
    for word in text.split():
        if word in stopwords.words('english'):
            new_list.append('')
        else:
            new_list.append(word)
    x = new_list[:]
    new_list.clear()
    return " ".join(x)

In [ ]:
stopword_removal('this is a good movie')

'   good movie'

In [ ]:
movie_df['review'].apply(stopword_removal)

,review
0,one reviewers mentioned watching 1 oz e...
1,wonderful little production. filming techniq...
2,thought wonderful way spend time hot s...
3,basically there's family little boy (jake) ...
4,"petter mattei's ""love time money"" visuall..."
...,...
95,daniel day-lewis versatile actor alive. eng...
96,guess would originally going least two ...
97,"well, like watch bad horror b-movies, cause ..."
98,"worst movie ever seen, well as, worst ..."


In [ ]:
def remove_emoji(text):
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', text)

In [ ]:
text3 = u'This dog \U0001f602'
print(text3)

This dog 😂


In [ ]:
remove_emoji(text3)

'This dog '

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 8.5 MB/s eta 0:00:00


In [ ]:
import emoji

In [ ]:
print(emoji.demojize('python is 🔥'))

python is :fire:


## tokenisation

In [ ]:
sentence1 = 'I am a data scientist'
sentence1.split()

['I', 'am', 'a', 'data', 'scientist']

In [ ]:
sentence2 = 'I am a data scientist. I like to code.'
sentence2.split('.')

['I am a data scientist', ' I like to code', '']

In [ ]:
#problems with split
sentence3 = 'I am a data scientist! I like to code. I also like to draw.'
sentence3.split('.')


['I am a data scientist! I like to code', ' I also like to draw', '']

In [ ]:
#tokenisation using regex
tokens = re.findall("[\w']+", sentence3)
tokens


['I',
 'am',
 'a',
 'data',
 'scientist',
 'I',
 'like',
 'to',
 'code',
 'I',
 'also',
 'like',
 'to',
 'draw']

In [ ]:
sentences =  re.findall("[A-Z][^\.!?]*[\.!?]", sentence3)
sentences

['I am a data scientist!', 'I like to code.', 'I also like to draw.']

### NLTK

In [ ]:
word_tokenize(sentence3)

['I',
 'am',
 'a',
 'data',
 'scientist',
 '!',
 'I',
 'like',
 'to',
 'code',
 '.',
 'I',
 'also',
 'like',
 'to',
 'draw',
 '.']

In [ ]:
sent_tokenize(sentence3)

['I am a data scientist!', 'I like to code.', 'I also like to draw.']

### Tokenisation using spacy

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
doc1 = nlp(sentence3)
doc1

I am a data scientist! I like to code. I also like to draw.

In [ ]:
for token in doc1:
    print(token)

I
am
a
data
scientist
!
I
like
to
code
.
I
also
like
to
draw
.


## Stemming

In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

In [ ]:
sample = "walk walks walking walked"
stem_words(sample)

'walk walk walk walk'

## Lemmatization

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
sentence = 'He was running and eating at the same time. He has a bad habit of swimming after playing!'
punctuations="?:!.,;"
sentence_words = nltk.word_tokenize(sentence)
for word in sentence_words:
    if word in punctuations:
        sentence_words.remove(word)

sentence_words

['He',
 'was',
 'running',
 'and',
 'eating',
 'at',
 'the',
 'same',
 'time',
 'He',
 'has',
 'a',
 'bad',
 'habit',
 'of',
 'swimming',
 'after',
 'playing']

In [ ]:
print("{0:20}{1:20}".format("Word","Lemma"))
for word in sentence_words:
    print ("{0:20}{1:20}".format(word,wordnet_lemmatizer.lemmatize(word, pos='v')))

Word                Lemma               
He                  He                  
was                 be                  
running             run                 
and                 and                 
eating              eat                 
at                  at                  
the                 the                 
same                same                
time                time                
He                  He                  
has                 have                
a                   a                   
bad                 bad                 
habit               habit               
of                  of                  
swimming            swim                
after               after               
playing             play                


## Bag of Words

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame({'text':['people watch campusx','campusx watch campusx','people write comment','campusx write comment'],
                   'output':[1,1,0,0]})

In [ ]:
df

,text,output
0,people watch campusx,1
1,campusx watch campusx,1
2,people write comment,0
3,campusx write comment,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [ ]:
bow = cv.fit_transform(df['text'])

In [ ]:
print(cv.vocabulary_)  #index numbers

{'people': 2, 'watch': 3, 'campusx': 0, 'write': 4, 'comment': 1}


In [ ]:
bow.toarray()

array([[1, 0, 1, 1, 0],
       [2, 0, 0, 1, 0],
       [0, 1, 1, 0, 1],
       [1, 1, 0, 0, 1]])

In [ ]:
cv.transform(['campusx watch and write comment of campusx']).toarray()

array([[2, 1, 0, 1, 1]])

In [ ]:
cv.transform(['campusx adkkj and write comment of campusx']).toarray()

array([[2, 1, 0, 0, 1]])

In [ ]:
X = bow.toarray()
y = df['output']

### N-grams

In [ ]:
cv = CountVectorizer(ngram_range=(2,2))

In [ ]:
bow = cv.fit_transform(df['text'])

In [ ]:
print(cv.vocabulary_)

{'people watch': 2, 'watch campusx': 4, 'campusx watch': 0, 'people write': 3, 'write comment': 5, 'campusx write': 1}


In [ ]:
bow.toarray()

array([[0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 0, 1]])

## TFIDF (Term Frequency Inverse Document Frequency)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
arr = tfidf.fit_transform(df['text']).toarray()

In [ ]:
arr

array([[0.49681612, 0.        , 0.61366674, 0.61366674, 0.        ],
       [0.8508161 , 0.        , 0.        , 0.52546357, 0.        ],
       [0.        , 0.57735027, 0.57735027, 0.        , 0.57735027],
       [0.49681612, 0.61366674, 0.        , 0.        , 0.61366674]])

## TEXT CLASSIFICATION

In [5]:
movie_df = movie_df[:10000]

In [6]:
movie_df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of v...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen-...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well b...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situ...",positive


In [38]:
movie_df['sentiment'].value_counts()

,count
sentiment,
positive,5028
negative,4972


In [11]:
movie_df.isnull().sum()

,0
review,0
sentiment,0


In [12]:
movie_df.duplicated().sum()

17

In [7]:
movie_df.drop_duplicates(inplace=True)

In [8]:
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [9]:
movie_df['review'] = movie_df['review'].apply(remove_html_tags)

In [10]:
movie_df['review'] = movie_df['review'].apply(lambda x: x.lower())

In [11]:
sw_list = stopwords.words('english')
movie_df['review'] = movie_df['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw_list)]))

In [44]:
movie_df['review'][0]

"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.i would say main appeal show due fact goes shows dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. violence, injustice (crooked guards who'll sold nickel, inmates who'll kill order g

In [14]:
X = movie_df['review']
y = movie_df['sentiment']

In [15]:
le = LabelEncoder()
y = le.fit_transform(y)

In [16]:
X.head()

,review
0,"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. ha..."
1,"wonderful little production. filming technique unassuming- old-time-bbc fashion gives comforting, sometimes discomforting, sense realism entire piece. actors extremely well chosen- michael sheen ""has got polari"" voices pat too! truly see seamless edit..."
2,"thought wonderful way spend time hot summer weekend, sitting air conditioned theater watching light-hearted comedy. plot simplistic, dialogue witty characters likable (even well bread suspected serial killer). may disappointed realize match point 2: r..."
3,"basically there's family little boy (jake) thinks there's zombie closet & parents fighting time.this movie slower soap opera... suddenly, jake decides become rambo kill zombie.ok, first going make film must decide thriller drama! drama movie watchable..."
4,"petter mattei's ""love time money"" visually stunning film watch. mr. mattei offers us vivid portrait human relations. movie seems telling us money, power success people different situations encounter. variation arthur schnitzler's play theme, director ..."


In [17]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [71]:
y_train.shape

(7986,)

In [72]:
X_train.shape

(7986,)

### Applying Bag of Words

In [73]:
cv = CountVectorizer()
X_train_bow = cv.fit_transform(X_train).toarray()
X_test_bow = cv.transform(X_test).toarray()

In [74]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [75]:
gnb  = GaussianNB()
gnb.fit(X_train_bow, y_train)

GaussianNB()

In [76]:
y_pred = gnb.predict(X_test_bow)

In [77]:
accuracy_score(y_test, y_pred)

0.6324486730095142

In [78]:
confusion_matrix(y_test, y_pred)

array([[717, 235],
       [499, 546]])

In [80]:
## Random forest classifier
rf = RandomForestClassifier()
rf.fit(X_train_bow, y_train)

RandomForestClassifier()

In [81]:
y_pred_rf= rf.predict(X_test_bow)

In [82]:
accuracy_score(y_test, y_pred_rf)

0.8467701552328493

In [83]:
cv = CountVectorizer(max_features = 3000)  #3000 frequent unique words
X_train_bow = cv.fit_transform(X_train).toarray()
X_test_bow = cv.transform(X_test).toarray()
rf = RandomForestClassifier()

rf.fit(X_train_bow, y_train)
y_pred_rf= rf.predict(X_test_bow)
accuracy_score(y_test, y_pred_rf)


0.8422633950926389

### N-gram

In [18]:
cv = CountVectorizer(ngram_range=(2,2), max_features = 3000) #not defining max feat led to system crash

In [19]:
X_train_bow = cv.fit_transform(X_train).toarray()
X_test_bow = cv.transform(X_test).toarray()
rf = RandomForestClassifier()

rf.fit(X_train_bow, y_train)
y_pred_rf= rf.predict(X_test_bow)
accuracy_score(y_test, y_pred_rf)

0.7356034051076615

### TFIDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [22]:
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()
rf = RandomForestClassifier()

rf.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [23]:
y_pred  = rf.predict(X_test_tfidf)
accuracy_score(y_test, y_pred)

0.8477716574862293